In [2]:
import subprocess

result = subprocess.run(
    ['echo', 'Hello from the child!'],
    capture_output=True,
    encoding='utf-8'
)
result.check_returncode()
result.stdout

'Hello from the child!\n'

In [8]:
import sys
sys.set_int_max_str_digits(10000)

In [12]:
proc = subprocess.Popen(['sleep', '2'])
while proc.poll() is None:
    print("working...")
    from time import sleep
    sleep(1)
    print("Done")

working...
Done
working...
Done


In [4]:
import time, subprocess
sleep_procs = []

start = time.time()
for _ in range(20):
    proc = subprocess.Popen(['sleep', '2'])
    sleep_procs.append(proc)
for proc in sleep_procs:
    proc.communicate()

In [6]:
import os, subprocess

def run_encrypt(data):
    env = os.environ.copy()
    env['password'] = '123456'
    proc = subprocess.Popen(['openssl', 'enc', '-des3','-pass', 'env:password'],
                            env=env,
                            stdin=subprocess.PIPE,
                            stdout=subprocess.PIPE)
    proc.stdin.write(data)
    proc.stdin.flush()
    return proc

procs = []

for _ in range(3):
    data = os.urandom(10)
    proc = run_encrypt(data)
    procs.append(proc)

for proc in procs:
    out, inp = proc.communicate()
    print(out, inp)

b'Salted__\xee\x82\x13gE\x17\xd6\xc9\xaf~\xde\x98N\xe6\x8f\xcf\xd9 \xa1\xec\x8d:\xca\x18' None
b'Salted__.?\x92\xbc\x14\xfa\x1bAMV\xa7v\xee\x1d\x94\xbc\xd7\xc3\x9c\x17\xa2\x1f\x1a\xf9' None
b'Salted__\xacOqF\xaa\xdc\xffs\x0e\xca\xf0\x13\x07\xd6\xa8j\xea\x7f\xd8\xaf\x91c\x9f\x8e' None


*** WARNING : deprecated key derivation used.
Using -iter or -pbkdf2 would be better.
*** WARNING : deprecated key derivation used.
Using -iter or -pbkdf2 would be better.
*** WARNING : deprecated key derivation used.
Using -iter or -pbkdf2 would be better.


In [29]:
import os, subprocess


def run_encrypt(data):
    env = os.environ.copy()
    env["password"] = "123456"
    proc = subprocess.Popen(
        ["openssl", "enc", "-des3", "-pass", "env:password"],
        env=env,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
    )
    proc.stdin.write(data)
    proc.stdin.flush()
    return proc


def run_hash(input_stdin):
    return subprocess.Popen(
        ["openssl", "dgst", "-sha256"],
        stdin=input_stdin,
        stdout=subprocess.PIPE,
    )


encrypt_procs = []
hash_procs = []

for _ in range(1):
    data = os.urandom(10)
    
    encrypt_proc = run_encrypt(data)
    encrypt_procs.append(encrypt_proc)

    hash_proc = run_hash(encrypt_proc.stdout)
    hash_procs.append(hash_proc)

    encrypt_proc.stdout.close()
    encrypt_proc.stdout = None

print("encrypt_procs", encrypt_procs)
for proc in encrypt_procs:
    proc.wait()
    # proc.wait()
    assert proc.returncode == 0

for proc in hash_procs:
    out, _ = proc.communicate(timeout=0.1)
    print(out.decode("latin-1"))
    assert proc.returncode == 0


*** WARNING : deprecated key derivation used.
Using -iter or -pbkdf2 would be better.


encrypt_procs [<Popen: returncode: None args: ['openssl', 'enc', '-des3', '-pass', 'env:pas...>]


KeyboardInterrupt: 